In [2]:
from keras.models import load_model
model = load_model("ppe_model.keras")

/Users/alfarezyyd/Projects/PythonProjects/ppe-detection/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
import cv2
import numpy as np

def preprocess_image(image_path, target_size=224):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (target_size, target_size))
    img = img / 255.0  # normalize jika saat training juga dilakukan
    img = np.expand_dims(img, axis=0)  # tambah batch dimensi
    return img

In [4]:
# Load gambar
image = preprocess_image("dataset/test/images/00113_jpg.rf.70816ad5893dab7fc737b7957616ee28.jpg")

# Inference
class_pred, bbox_pred = model.predict(image)

# class_pred bentuknya array dari softmax, ambil kelas tertinggi
predicted_class = np.argmax(class_pred)
confidence = np.max(class_pred)

# bbox_pred hasilnya koordinat [x_center, y_center, width, height] (diasumsikan seperti saat training)
bbox = bbox_pred[0]  # karena satu gambar
print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Predicted Class: 7, Confidence: 1.00


In [5]:
# Convert dari rasio ke pixel (jika perlu)
# img = cv2.imread("dataset/test/images/00113_jpg.rf.70816ad5893dab7fc737b7957616ee28.jpg")
# h, w, _ = img.shape
# x_center, y_center, bw, bh = bbox
# x1 = int((x_center - bw / 2) * w)
# y1 = int((y_center - bh / 2) * h)
# x2 = int((x_center + bw / 2) * w)
# y2 = int((y_center + bh / 2) * h)

# # Gambar bounding box dan label
# label = f"Class: {predicted_class} ({confidence:.2f})"
# cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
# cv2.putText(img, label, (x1, y1 - 10),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# # Tampilkan gambar
# cv2.imshow("Prediction", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [10]:
import os

test_image_dir = 'dataset/test/images'
test_label_dir = 'dataset/test/labels'
output_dir = 'output_predictions'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(test_image_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_image_dir, filename)

        # 1. Load gambar asli untuk ditampilkan (jangan pakai img_input)
        original_img = cv2.imread(image_path)
        original_h, original_w = original_img.shape[:2]

        # 2. Preprocess gambar untuk model
        img_input = cv2.resize(original_img, (224, 224)) / 255.0
        img_input = np.expand_dims(img_input, axis=0)

        # 3. Prediksi
        class_pred, bbox_pred = model.predict(img_input)
        predicted_class = np.argmax(class_pred[0])
        confidence = np.max(class_pred[0])
        x_center, y_center, w, h = bbox_pred[0]

        # 4. Skala ulang koordinat dari 224x224 ke ukuran gambar asli
        x_center *= original_w
        y_center *= original_h
        w *= original_w
        h *= original_h

        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)

        # 5. Gambar bounding box dan label
        cv2.rectangle(original_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"Class {predicted_class} ({confidence:.2f})"
        cv2.putText(original_img, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 6. Simpan hasil
        cv2.imwrite(os.path.join(output_dir, filename), original_img)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━

NameError: name 'img' is not defined